# 🤖 Agente de Análise de Sentimento de Notícias

Este notebook implementa um agente inteligente que:
1. Busca notícias usando SerpAPI
2. Analisa o sentimento de cada notícia usando Mistral AI
3. Apresenta os resultados de forma estruturada

---

## 📦 Passo 1: Instalar Dependências

Execute apenas uma vez ou se precisar atualizar os pacotes.

In [1]:
!pip install python-dotenv langchain-mistralai langchain-core google-search-results langgraph


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


## 🔑 Passo 2: Configurar Variáveis de Ambiente

**Opção 1:** Criar arquivo `.env` na mesma pasta do notebook:
```env
MISTRAL_API_KEY=sua_chave_mistral
SERPAPI_API_KEY=sua_chave_serpapi
```

**Opção 2:** Definir diretamente no código:

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

print("✅ MISTRAL_API_KEY:", "Configurada" if os.getenv("MISTRAL_API_KEY") else "❌ NÃO ENCONTRADA")
print("✅ SERPAPI_API_KEY:", "Configurada" if os.getenv("SERPAPI_API_KEY") else "❌ NÃO ENCONTRADA")

✅ MISTRAL_API_KEY: Configurada
✅ SERPAPI_API_KEY: Configurada


## 📚 Passo 3: Importar Bibliotecas

In [3]:
import time
from langchain_mistralai import ChatMistralAI
from langchain_core.tools import tool
from serpapi import GoogleSearch
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional, List, Dict

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


## 🧠 Passo 4: Configurar o Modelo LLM

In [4]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    api_key=os.getenv("MISTRAL_API_KEY"),
    temperature=0
)

print("✅ Modelo Mistral configurado!")

✅ Modelo Mistral configurado!


## 🛠️ Passo 5: Definir Ferramentas (Tools)

In [5]:
@tool
def google_search(query: str):
    """Busca notícias usando SerpAPI."""
    params = {
        "engine": "google",
        "q": query,
        "api_key": os.getenv("SERPAPI_API_KEY")
    }
    result = GoogleSearch(params).get_dict()
    return result


@tool
def sentiment_analysis(text: str):
    """Classifica o sentimento do texto."""
    prompt = f"""
    Classifique o sentimento do texto abaixo como Positivo, Negativo ou Neutro.
    Explique brevemente a razão.
    Responda APENAS em formato JSON válido com as chaves: "label" e "reason".

    Texto:
    {text}
    """

    resp = llm.invoke(prompt)
    return resp.content

print("✅ Ferramentas definidas!")

✅ Ferramentas definidas!


## 📊 Passo 6: Definir o Estado do Agente

In [6]:
class AgentState(TypedDict):
    query: str
    results: Optional[dict]
    analyzed: Optional[List[Dict]]

print("✅ Estado do agente definido!")

✅ Estado do agente definido!


## 🔍 Passo 7: Implementar Função de Busca

In [ ]:
def step_search(state: AgentState):
    """Executa a busca no Google."""
    query = state["query"]
    print(f"📡 Executando busca para: '{query}'")
    
    api_key = os.getenv("SERPAPI_API_KEY")
    
    if not api_key:
        print("❌ ERRO: SERPAPI_API_KEY não encontrada no arquivo .env")
        return {"results": {}}
    
    print(f"✅ API Key encontrada: {api_key[:10]}...")
    
    try:
        data = GoogleSearch({
            "engine": "google",
            "q": query,
            "api_key": api_key
        }).get_dict()
        
        if 'error' in data:
            print(f"❌ ERRO DA API SERPAPI: {data['error']}")
            return {"results": {}}
        
        print(f"✅ Busca concluída! Resultados encontrados: {len(data.get('organic_results', []))}")
        
        if data.get('organic_results'):
            print(f"✅ Primeira notícia: {data['organic_results'][0].get('title', 'Sem título')}")
        else:
            print("⚠️  Nenhum resultado em 'organic_results'")
            print(f"🔍 Keys disponíveis no retorno: {list(data.keys())}")
        
        return {"results": data}
    
    except Exception as e:
        print(f"❌ ERRO na busca: {e}")
        return {"results": {}}

print("✅ Função de busca implementada!")

✅ Função de busca implementada!


## 🧪 Passo 8: Implementar Função de Análise de Sentimento

In [ ]:
def step_analyze(state: AgentState):
    """Analisa o sentimento das notícias."""
    print("\n🧠 Iniciando análise de sentimento...")
    
    analyzed = []

    results = state.get("results", {})
    news_items = results.get("organic_results", [])
    
    print(f"📊 Total de notícias para analisar: {len(news_items)}")
    
    if not news_items:
        print("⚠️  AVISO: Nenhuma notícia encontrada para analisar!")
        print(f"🔍 Conteúdo de 'results': {list(results.keys()) if results else 'Vazio'}")
        return {"analyzed": []}
    
    for i, item in enumerate(news_items[:3], 1):  # analisa apenas 3 notícias
        title = item.get("title", "")
        if title:
            print(f"\n  ⏳ Analisando notícia {i}/3: {title[:50]}...")
            
            prompt = f"""
            Classifique o sentimento do texto abaixo como Positivo, Negativo ou Neutro.
            Explique brevemente a razão.
            Responda APENAS em formato JSON válido com as chaves: "label" e "reason".

            Texto:
            {title}
            """
            
            try:
                max_retries = 3
                for attempt in range(max_retries):
                    try:
                        sent = llm.invoke(prompt)
                        analyzed.append({
                            "title": title,
                            "sentiment": sent.content
                        })
                        print(f"  ✅ Análise {i} concluída!")
                        break
                    except Exception as retry_error:
                        if "429" in str(retry_error) and attempt < max_retries - 1:
                            wait_time = (2 ** attempt) * 2  # 2s, 4s, 8s
                            print(f"  ⚠️  Rate limit! Tentativa {attempt + 1}/{max_retries}. Aguardando {wait_time}s...")
                            time.sleep(wait_time)
                        else:
                            raise
                
                if i < 3:
                    time.sleep(2)
                    
            except Exception as e:
                print(f"  ❌ Erro ao analisar notícia {i} após {max_retries} tentativas: {str(e)[:100]}")

    print(f"\n✅ Análise concluída! Total analisado: {len(analyzed)}")
    return {"analyzed": analyzed}

print("✅ Função de análise implementada!")

✅ Função de análise implementada!


## 🔗 Passo 9: Montar o Grafo do Agente

In [9]:
graph = StateGraph(AgentState)

graph.add_node("search", step_search)
graph.add_node("analyze", step_analyze)

graph.set_entry_point("search")
graph.add_edge("search", "analyze")
graph.add_edge("analyze", END)

agent = graph.compile()

print("✅ Grafo do agente montado e compilado!")

✅ Grafo do agente montado e compilado!


## 🚀 Passo 10: Executar o Agente

Escolha um tema abaixo ou personalize sua busca:

In [ ]:
query = "cura do cancer"                   
# query = "assassinatos"                

print(f"\n🔍 Buscando: {query}\n")
print("="*70)


🔍 Buscando: cura do cancer



In [11]:
try:
    result = agent.invoke({"query": query})

    print("\n" + "="*70)
    print("=== RESULTADO FINAL ===")
    print("="*70)
    
    if result.get("analyzed"):
        for i, item in enumerate(result["analyzed"], 1):
            print(f"\n{'='*70}")
            print(f"📰 Notícia {i}")
            print(f"{'='*70}")
            print(f"Título: {item['title']}")
            print(f"\nSentimento:\n{item['sentiment']}")
    else:
        print("❌ Nenhum resultado encontrado.")

except Exception as e:
    print(f"\n❌ Erro ao executar o agente: {e}")

📡 Executando busca para: 'cura do cancer'
✅ API Key encontrada: 37056e36cc...
✅ Busca concluída! Resultados encontrados: 9
✅ Primeira notícia: ¿Se puede curar el cáncer? | ¿El tratamiento ...

🧠 Iniciando análise de sentimento...
📊 Total de notícias para analisar: 9

  ⏳ Analisando notícia 1/3: ¿Se puede curar el cáncer? | ¿El tratamiento ......
  ✅ Análise 1 concluída!

  ⏳ Analisando notícia 2/3: Existem curas para o câncer que foram escondidas d...
  ✅ Análise 2 concluída!

  ⏳ Analisando notícia 3/3: É possível falar em cura? - Instituto Nacional de ...
  ⚠️  Rate limit! Tentativa 1/3. Aguardando 2s...
  ⚠️  Rate limit! Tentativa 2/3. Aguardando 4s...
  ❌ Erro ao analisar notícia 3 após 3 tentativas: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"object":"error","mes

✅ Análise concluída! Total analisado: 2

=== RESULTADO FINAL ===

📰 Notícia 1
Título: ¿Se puede curar el cáncer? | ¿El tratamiento ...

Sentimento:
```json
{
  "label": "Neutro",
  "re

## 📊 Passo 11: Visualizar Resultados de Forma Resumida

In [12]:
import json

if result.get("analyzed"):
    print("\n📊 RESUMO DOS SENTIMENTOS:\n")
    
    sentimentos = {"Positivo": 0, "Negativo": 0, "Neutro": 0}
    
    for item in result["analyzed"]:
        try:
            # Tenta extrair o JSON da resposta
            sent_text = item['sentiment'].replace('```json', '').replace('```', '').strip()
            sent_data = json.loads(sent_text)
            label = sent_data.get('label', 'Neutro')
            
            if label in sentimentos:
                sentimentos[label] += 1
        except:
            sentimentos["Neutro"] += 1
    
    total = sum(sentimentos.values())
    
    for sent, count in sentimentos.items():
        emoji = "🟢" if sent == "Positivo" else "🔴" if sent == "Negativo" else "⚪"
        percent = (count/total)*100 if total > 0 else 0
        print(f"{emoji} {sent}: {count} notícia(s) ({percent:.1f}%)")
else:
    print("Nenhum resultado para resumir.")


📊 RESUMO DOS SENTIMENTOS:

🟢 Positivo: 0 notícia(s) (0.0%)
🔴 Negativo: 1 notícia(s) (50.0%)
⚪ Neutro: 1 notícia(s) (50.0%)
